In [1]:
# pip install --upgrade torch torchvision torchaudio --no-cache-dir


In [1]:
from depth_kosmos import requests, DepthKosmosCaptioner
from depth_kosmos import Image as Image_PIL

depth_kosmos_captioner = DepthKosmosCaptioner()


Current directory: /root/Depth-SoM/Depth-Anything-V2


xFormers not available
xFormers not available
2025-04-04 18:13:42.149439: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 18:13:42.164403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743790422.181899    1979 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743790422.187167    1979 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743790422.200269    1979 computation_placer.cc:177] computation placer already regis

Using device: cuda


preprocessor_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

config.json:   0%|          | 0.00/4.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

depth_anything_v2_vitl.pth:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Using device: cuda


In [2]:
# !pip install numpy --upgrade


In [3]:

url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTp5jMLHnfiO56w8iVWAwI4VvOu4B_5c2C1ww&s"
image = Image_PIL.open(requests.get(url, stream=True).raw)


In [4]:
# depth_kosmos_captioner.display_depth_images(image)
full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
print(full_caption_string)

Closest: a woman carrying a basket of plants
----
Mid Range: a soldier
----
Farthest: a woman
----



In [ ]:
import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file


# --- Configuration ---

hf_token = os.getenv("HF_TOKEN")

hf_user = "Rajarshi-Roy-research"
# dataset_name = "Flickr30k_Grounding_Som"
# commit_message = "Updating dataset with image paths"
# img_cols = ["image","wbox_image"]
# image_to_process = "image"

dataset_name = "refcocog-eval-depth"
commit_message = "Updating dataset with images and depth caption"
img_cols = ["image"]
image_to_process = "image"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data and returns the saved file path."""
    try:
        image_bytes = image_data.get("bytes", None)
        if image_bytes:
            image = Image_PIL.open(io.BytesIO(image_bytes))
            image_path = os.path.join(image_folder, f"image_{idx}.jpg")
            image.save(image_path, format="JPEG")
            print(f"✅ Saved: {image_path}")
            return image_path
        else:
            print(f"⚠️ Skipped row {idx}: No bytes found")
            return None
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None

def process_image_from_df(df, depth_kosmos_captioner, image_folder):
    """Processes the DataFrame, adding depth captions and saving progress."""
    if "depth_caption" not in df.columns:
        df["depth_caption"] = None

    for index, row in tqdm(df.iterrows(), total=len(df)):
        if row["depth_caption"] is None:
            try:
                if isinstance(row["image"], dict): #Handle case where image is still a dict
                    image_path = save_image_from_bytes(row["image"], index, image_folder)
                    if image_path:
                        image = Image_PIL.open(image_path)
                        full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                        df.loc[index, "depth_caption"] = full_caption_string
                elif isinstance(row["image"], str): #Handle case where image is a file path
                    image = Image_PIL.open(row["image"])
                    full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                    df.loc[index, "depth_caption"] = full_caption_string
                else:
                    print(f"⚠️ Unexpected image type at index {index}: {type(row['image'])}")
                    df.loc[index, "depth_caption"] = "Error: Unexpected image type"

                df.to_parquet("df_with_captions.parquet")
            except Exception as e:
                print(f"Error processing image at index {index}: {e}")
                df.loc[index, "depth_caption"] = "Error"
                df.to_parquet("df_with_captions.parquet")

    return df

def push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=False):
    """Pushes the DataFrame to the Hugging Face Hub."""
    hf_dataset = Dataset.from_pandas(df)
    
    for img_col in img_cols:
        hf_dataset = hf_dataset.cast_column(img_col, Image())

    try:
        hf_dataset.push_to_hub(
            repo_id=f"{hf_user}/{dataset_name}",
            commit_message=commit_message,
            token=hf_token,
            private=private,
        )
        print(f"✅ Dataset successfully uploaded: {hf_user}/{dataset_name}")
    except Exception as e:
        print(f"❌ Error pushing to Hugging Face Hub: {e}")


# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
df = ds["train"].to_pandas()

for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)

df.to_csv("updated_dataset.csv", index=False)
print("✅ Updated DataFrame saved as 'updated_dataset.csv'!")

#Process and add captions
try:
    df = pd.read_csv("updated_dataset.csv")
except FileNotFoundError:
    pass

df = process_image_from_df(df, depth_kosmos_captioner, image_to_process)

# Authenticate and push to Hugging Face

login(token=hf_token)
push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=True)

df


README.md:   0%|          | 0.00/715 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/60.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

✅ Saved: image/image_0.jpg
✅ Saved: image/image_1.jpg
✅ Saved: image/image_2.jpg
✅ Saved: image/image_3.jpg
✅ Saved: image/image_4.jpg
✅ Saved: image/image_5.jpg
✅ Saved: image/image_6.jpg
✅ Saved: image/image_7.jpg
✅ Saved: image/image_8.jpg
✅ Saved: image/image_9.jpg
✅ Saved: image/image_10.jpg
✅ Saved: image/image_11.jpg
✅ Saved: image/image_12.jpg
✅ Saved: image/image_13.jpg
✅ Saved: image/image_14.jpg
✅ Saved: image/image_15.jpg
✅ Saved: image/image_16.jpg
✅ Saved: image/image_17.jpg
✅ Saved: image/image_18.jpg
✅ Saved: image/image_19.jpg
✅ Saved: image/image_20.jpg
✅ Saved: image/image_21.jpg
✅ Saved: image/image_22.jpg
✅ Saved: image/image_23.jpg
✅ Saved: image/image_24.jpg
✅ Saved: image/image_25.jpg
✅ Saved: image/image_26.jpg
✅ Saved: image/image_27.jpg
✅ Saved: image/image_28.jpg
✅ Saved: image/image_29.jpg
✅ Saved: image/image_30.jpg
✅ Saved: image/image_31.jpg
✅ Saved: image/image_32.jpg
✅ Saved: image/image_33.jpg
✅ Saved: image/image_34.jpg
✅ Saved: image/image_35.jpg
✅ 

100%|██████████| 400/400 [11:18<00:00,  1.70s/it]


❌ Error pushing to Hugging Face Hub: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-67f02457-5a998eb87aa4e42e295d5591;0c073dba-de67-4467-a6c3-203544cb3459)

Invalid username or password.


,id,caption,bbox,bbox_area,bbox_id,ori_category_id,image_id,height,width,file_name,is_rewrite,split,image,depth_caption
0,4381,A pair of tortillas containing a visible filli...,[300.25585936 239.42510986 183.92150884 133.25...,24508.697297,o365_24196196,o365_172,o365_489077,512,769,sampled_images/objects365_v1_00489077.jpg,True,val,image/image_0.jpg,Closest: a couple\n----\nMid Range: a bar\n---...
1,3385,"The closed-top, white bucket-shaped item, posi...",[647.26123049 556.64379886 98.25061038 106.61...,10474.736003,o365_3289057,o365_49,o365_1040341,768,1024,sampled_images/objects365_v2_01040341.jpg,True,val,image/image_1.jpg,Closest: a man\n----\nMid Range: tents\n----\n...
2,27674,The pink and white surfboard is curretnly ride...,[120.05 163.64 232.97 77.18],17980.624600,coco_650459,refcoco_42,coco_449136,318,640,sampled_images/COCO_train2014_000000449136.jpg,True,val,image/image_2.jpg,Closest: a man\n----\nMid Range: a surfer\n---...
3,44294,"The man wearing a cheerful Santa hat, characte...",[240.81 14.21 275.38 370.15],101931.907000,coco_2165449,refcoco_1,coco_242583,427,640,sampled_images/COCO_train2014_000000242583.jpg,True,val,image/image_3.jpg,Closest: a man\n----\nMid Range: a hand\n----\...
4,29717,The pizza adorned with chunks of white topping...,[143.32 283.76 353.97 139.47],49368.195900,coco_1074038,refcoco_59,coco_70415,429,640,sampled_images/COCO_train2014_000000070415.jpg,False,val,image/image_4.jpg,Closest: a pizza\n----\nMid Range: a person\n-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12011,The green purse hanging on the wall close to t...,[232.43835453 416.28540035 57.75231928 61.99...,3580.129160,o365_27275769,o365_14,o365_2053317,982,1024,sampled_images/objects365_v2_02053317.jpg,True,val,image/image_395.jpg,Closest: two motorcycles\n----\nMid Range: a m...
396,36631,"The man standing in the background, behind the...",[363.87 211.18 116.49 306.33],35684.381700,coco_500529,refcoco_1,coco_242213,640,512,sampled_images/COCO_train2014_000000242213.jpg,False,val,image/image_396.jpg,Closest: a baseball pitcher\n----\nMid Range: ...
397,18231,The skier clad in a red jacket is standing wit...,[474.02 105.55 130.5 316.65],41322.825000,coco_512212,refcoco_1,coco_271641,427,640,sampled_images/COCO_train2014_000000271641.jpg,False,val,image/image_397.jpg,Closest: people\n----\nMid Range: a silhouette...
398,5846,A transparent drinking vessel containing a yel...,[284.04260255 188.13116452 100.05944822 199.01...,19913.766167,o365_9095135,o365_11,o365_476705,683,512,sampled_images/objects365_v1_00476705.jpg,False,val,image/image_398.jpg,Closest: candles\n----\nMid Range: a plate of ...


Token has not been saved to git credential helper.


In [7]:

login(token=hf_token)
push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=True)


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

✅ Dataset successfully uploaded: Rajarshi-Roy-research/refcocog-eval-depth
